In [1]:
# import the required packages
import numpy as np
import pandas as pd
from tensorflow import keras
from flask import Flask, request, jsonify, render_template
import pubchempy as pcp
import pickle
from keras import backend
from padelpy import from_smiles

Using TensorFlow backend.


In [2]:
# loading built models and features
def loading_model():
    models = [keras.models.load_model(ele+'_model.h5', compile=False) for ele in ['E',"S","A","B"]]
    Scaler = pickle.load(open("Scaler.pkl",'rb'))
    feature_name = list(pickle.load(open("feature_name.pkl",'rb')))
    return models,Scaler,feature_name

In [3]:
# generating the input features and do scaling
def generate_feature(SMILES,feature_name,Scaler):

    descriptors = from_smiles(SMILES)
    Inform = pcp.compounds_to_frame(pcp.get_compounds(SMILES,'smiles'), properties=['xlogp','synonyms'])
    cids = pcp.get_cids(SMILES, 'smiles')
    V = [round(float(descriptors["McGowan_Volume"]),2)]
    features = map(lambda x: descriptors[x], feature_name)
    features = Scaler.transform(np.array(list(features)).reshape(1, -1))
    return features, V,Inform, cids

In [18]:
# making prediction for using the loaded models

def making_prediction(models,features,V,Inform):
    prediction = [round(float(model.predict(features)),2) for model in models] + V
    Surrogates = abs(np.dot([0.562,-1.054,0.034,-3.46,3.814],prediction[0:5]) +0.088-Inform["xlogp"].values)
    goodness = "G" if Surrogates<=0.5 else ("F" if Surrogates<=1 else "B")
    prediction.append(goodness)
    return prediction

In [19]:
SMILES = ["Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl",
"CSc1ccc2Sc3ccccc3N(CCC4CCCCN4C)c2c1",
"ClC(Cl)C(c1ccc(Cl)cc1)c2ccccc2Cl",
"Cc1cc(C)c2nc3ccc4ccccc4c3cc2c1",
"c1ccc2c(c1)ccc3cc4c(ccc5ccccc45)cc23",
"Cc1ccc2cc3c(ccc4ccccc34)c5CCc1c25",
"ClC1CC2C(C1Cl)C3(Cl)C(=C(Cl)C2(Cl)C3(Cl)Cl)Cl",
"Cc1c2ccccc2c(C)c3c4ccccc4ccc13"
]

In [22]:
new_prediction =[]
models,Scaler,feature_name = loading_model()
for smiles in SMILES:
   features, V,Inform, cids =generate_feature(smiles,feature_name,Scaler)
   prediction = making_prediction(models,features,V,Inform)
   new_prediction.append([list(Inform['synonyms'])[0][0].title()]+[smiles]+cids+prediction)
results = pd.DataFrame(new_prediction,columns=["Name","SMILES","CID","E","S","A","B","V","Goodness"])
results.head()
#results.to_csv("./results.csv")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


,Name,SMILES,CID,E,S,A,B,V,Goodness
0,Clofenotane,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,3036,1.76,1.65,0.01,0.15,2.22,G
1,Thioridazine,CSc1ccc2Sc3ccccc3N(CCC4CCCCN4C)c2c1,5452,2.84,2.05,0.14,1.22,2.90,G
2,Mitotane,ClC(Cl)C(c1ccc(Cl)cc1)c2ccccc2Cl,4211,1.77,1.51,0.02,0.31,2.10,G
3,"8,10-Dimethylbenz(A)Acridine",Cc1cc(C)c2nc3ccc4ccccc4c3cc2c1,66127,3.00,1.81,0.04,0.57,2.06,G
4,"Dibenz[A,H]Anthracene",c1ccc2c(c1)ccc3cc4c(ccc5ccccc45)cc23,5889,3.82,2.08,0.01,0.43,2.19,G


In [15]:
results.head()


,Name,SMILES,CID,E,S,A,B,V,Goodness
0,Clofenotane,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,3036,1.76,1.65,0.01,0.15,2.22,[2.22]
1,Thioridazine,CSc1ccc2Sc3ccccc3N(CCC4CCCCN4C)c2c1,5452,2.84,2.05,0.14,1.22,2.90,[2.9]
2,Mitotane,ClC(Cl)C(c1ccc(Cl)cc1)c2ccccc2Cl,4211,1.77,1.51,0.02,0.31,2.10,[2.1]
3,"8,10-Dimethylbenz(A)Acridine",Cc1cc(C)c2nc3ccc4ccccc4c3cc2c1,66127,3.00,1.81,0.04,0.57,2.06,[2.06]
4,"Dibenz[A,H]Anthracene",c1ccc2c(c1)ccc3cc4c(ccc5ccccc45)cc23,5889,3.82,2.08,0.01,0.43,2.19,[2.19]
